# Build a SageMaker Pipeline to train and register the injury narrative BERT classifier

In [1]:
%%capture
!pip install tensorflow
!pip install transformers
!pip install nltk
!pip install -U sagemaker

In [ ]:

subprocess.check_call([sys.executable, "-m", "conda", "install", "-c", "anaconda", "tensorflow==2.3.0", "-y"])
import tensorflow as tf
from tensorflow import keras

subprocess.check_call([sys.executable, "-m", "conda", "install", "-c", "conda-forge", "transformers==3.5.1", "-y"])
from transformers import BertTokenizer
from transformers import BertConfig

subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib==3.2.1"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "sagemaker==2.24.1"])


In [2]:
import pandas as pd
import tensorflow as tf
import re
import nltk
import string
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras import activations, optimizers, losses
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import numpy as np
import sagemaker
from sagemaker import get_execution_role
import joblib 
import collections

In [3]:
print(tf.__version__)

2.6.0


In [4]:

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
bucket = default_bucket
bucket

'sagemaker-us-east-1-979294212144'

In [5]:
role

'arn:aws:iam::979294212144:role/service-role/AmazonSageMaker-ExecutionRole-20210423T122185'

In [6]:
#%%time
#%run ./src/pre-processing.py --data_path ./data/raw  --train_percentage 1

In [7]:
#%run ./src/evaluate_model_metrics.py --input_data './data/test' \
#            --input_model './output/model/training-BaseBERT-08-02-58-54-2021-08-02-19-58-55-017' \
#            --max_len 45 \
#            --output_data './output/model'

In [8]:
num_records = 138549
num_valid_records = 15395
max_len = 45
epochs = 5
batch_size = 16
valid_batch_size = 16
steps_per_epoch = num_records // batch_size
validation_steps = num_valid_records // valid_batch_size
learning_rate = 5e-5
optimizer = 'adam'

In [9]:
print(num_records)
print(steps_per_epoch)
print(validation_steps)

138549
8659
962


In [8]:
%%time
#%run ./src/train.py --train ./data/train --validation ./data/valid --epochs 5 --num_records 138549 --steps_per_epoch 8659 --validation_steps 962

input train:  ./data/train
input valid:  ./data/valid
loading data...
train_dir :  ./data/train
train_file :  ./data/train/train.tfrecord
valid_dir: ./data/valid
valid_file: ./data/valid/valid.tfrecord
loading encoder...
Building model...
[2021-08-20 13:34:04.000 tensorflow-2-3-gpu--ml-g4dn-xlarge-c85184389676cdfa7bdf06745c9b:69 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-20 13:34:04.025 tensorflow-2-3-gpu--ml-g4dn-xlarge-c85184389676cdfa7bdf06745c9b:69 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 45)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 45)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 43, 512)      1180160     bert[0][0]            

8659/8659 [==============================] - 2472s 285ms/step - loss: 0.6467 - acc: 0.8075 - val_loss: 0.5283 - val_acc: 0.8363
Epoch 2/5
8659/8659 [==============================] - 2476s 286ms/step - loss: 0.2849 - acc: 0.9075 - val_loss: 0.5380 - val_acc: 0.8467
Epoch 3/5
8659/8659 [==============================] - 2480s 286ms/step - loss: 0.1613 - acc: 0.9480 - val_loss: 0.5681 - val_acc: 0.8442
Epoch 4/5
8659/8659 [==============================] - 2481s 287ms/step - loss: 0.1400 - acc: 0.9551 - val_loss: 0.5681 - val_acc: 0.8442
Epoch 5/5
8659/8659 [==============================] - 2479s 286ms/step - loss: 0.1380 - acc: 0.9554 - val_loss: 0.5681 - val_acc: 0.8442
saving model...
CPU times: user 2h 6min 26s, sys: 55min 53s, total: 3h 2min 20s
Wall time: 3h 28min 20s


### pipeline name

In [351]:
import time
timestamp = int(time.time())

pipeline_name = 'BaseBERT-Injury-Coding-pipeline-{}'.format(timestamp)

## Step 1 - Dataset and preprocessing step

## Upload raw dataset

In [352]:
prefix = 'injury-data/raw'
input_data_train = sagemaker_session.upload_data(path = './data/raw',
                                                      bucket = bucket,
                                                      key_prefix = prefix)
input_data_train

's3://sagemaker-us-east-1-979294212144/injury-data/raw'

## Configure pre-processing step

In [353]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

In [354]:
# 7 params
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.c5.2xlarge"
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

train_percentage = ParameterFloat(
    name="TrainPercentage",
    default_value=1.0
)

is_sample_dataset = ParameterString(
    name="SampleDataset",
    default_value="True"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_train
)

transformer_model = ParameterString(
    name="TransformerModel",
    default_value='bert-base-uncased'
)
max_seq_length = ParameterInteger(
    name="MaxSeqLength",
    default_value=45
)

In [355]:
region = sagemaker_session.boto_region_name

In [356]:
processing_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4.1",
    py_version="py37",
    instance_type=processing_instance_type,
    image_scope="training"
)
print(processing_image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.4.1-cpu-py37


In [357]:
from sagemaker.sklearn.processing import SKLearnProcessor,ScriptProcessor

processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={'AWS_DEFAULT_REGION': sagemaker_session.boto_region_name},     
     max_runtime_in_seconds=18000
)

In [358]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


processing_inputs=[
    ProcessingInput(
        input_name='raw-input-data',
        source=input_data,
        destination='/opt/ml/processing/input/data/',
        s3_data_distribution_type='ShardedByS3Key'
    )
]

processing_outputs=[
    ProcessingOutput(output_name='processed-train',
                     source='/opt/ml/processing/output/processed/train',
                     s3_upload_mode='EndOfJob'),
    ProcessingOutput(output_name='processed-validation',
                     source='/opt/ml/processing/output/processed/validation',
                     s3_upload_mode='EndOfJob'),
    ProcessingOutput(output_name='processed-test',
                     source='/opt/ml/processing/output/processed/test',
                     s3_upload_mode='EndOfJob'),
]        

processing_step = ProcessingStep(
    name='Pre-Processing', 
    code='./src/pre-processing.py',
    processor=processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    job_arguments=['--train_percentage', str(train_percentage.default_value),                   
                   '--max_len',str(max_seq_length.default_value),
                   '--transformer_model',str(transformer_model.default_value)
                  ]
)        

print(processing_step)

ProcessingStep(name='Pre-Processing', step_type=<StepTypeEnum.PROCESSING: 'Processing'>, depends_on=None)


In [359]:
import json

# print out the list of the processing job properties
print(json.dumps(
    processing_step.properties.__dict__,
    indent=4, sort_keys=True, default=str
))

{
    "AppSpecification": "<sagemaker.workflow.properties.Properties object at 0x7f9d6032bcd0>",
    "AutoMLJobArn": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b759490>",
    "CreationTime": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b759410>",
    "Environment": "<sagemaker.workflow.properties.Properties object at 0x7f9d6032be50>",
    "ExitMessage": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b7592d0>",
    "ExperimentConfig": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b7590d0>",
    "FailureReason": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b759310>",
    "LastModifiedTime": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b7593d0>",
    "MonitoringScheduleArn": "<sagemaker.workflow.properties.Properties object at 0x7f9d5b759450>",
    "NetworkConfig": "<sagemaker.workflow.properties.Properties object at 0x7f9d6032bf50>",
    "ProcessingEndTime": "<sagemaker.workflow.properties.Properties

In [360]:
print(json.dumps(
    processing_step.properties.ProcessingOutputConfig.Outputs['processed-train'].__dict__, 
    indent=4, sort_keys=True, default=str
))

{
    "AppManaged": "<sagemaker.workflow.properties.Properties object at 0x7f9d60c5cf50>",
    "FeatureStoreOutput": "<sagemaker.workflow.properties.Properties object at 0x7f9d60c5c410>",
    "OutputName": "<sagemaker.workflow.properties.Properties object at 0x7f9d60c5cc10>",
    "S3Output": "<sagemaker.workflow.properties.Properties object at 0x7f9d60c5cd90>",
    "_path": "Steps.Pre-Processing.ProcessingOutputConfig.Outputs['processed-train']",
    "_shape_names": [
        "ProcessingOutput"
    ]
}


In [361]:
print(json.dumps(
    processing_step.properties.ProcessingOutputConfig.Outputs['processed-train'].S3Output.S3Uri.__dict__,
    indent=4, sort_keys=True, default=str
))

{
    "__str__": "S3Uri",
    "_path": "Steps.Pre-Processing.ProcessingOutputConfig.Outputs['processed-train'].S3Output.S3Uri",
    "_shape_names": [
        "S3Uri"
    ]
}


## Step 3 - Training Step

In [362]:
# 12 params

epochs = ParameterInteger(
    name="Epochs",
    default_value=3
)

num_records = ParameterInteger(
    name="NumRecords",
    default_value = 138549
)
   

learning_rate = ParameterFloat(
    name="LearningRate",
    default_value=5e-5
) 
    
train_batch_size = ParameterInteger(
    name="TrainBatchSize",
    default_value=16
)

train_steps_per_epoch = ParameterInteger(
    name="TrainStepsPerEpoch",
    default_value=500
)

validation_batch_size = ParameterInteger(
    name="ValidationBatchSize",
    default_value=16
)

validation_steps_per_epoch = ParameterInteger(
    name="ValidationStepsPerEpoch",
    default_value=500
)


train_instance_count = ParameterInteger(
    name="TrainInstanceCount",
    default_value=1
)

train_instance_type = ParameterString(
    name="TrainInstanceType",
    default_value="ml.p3.2xlarge"
)


max_seq_length = ParameterInteger(
    name="MaxSeqLength",
    default_value=45
)

optimizer = ParameterString(
    name="optimizer",
    default_value='Adam'
)

input_mode = ParameterString(
    name="InputMode",
    default_value="File"
)

In [363]:
train_batch_size.default_value

16

In [364]:
hyperparameters={
    'max_seq_length': str(max_seq_length.default_value),
    'epochs': str(epochs.default_value),
    'num_records': str(num_records.default_value),
    'learning_rate': str(learning_rate.default_value),
    'batch_size': str(train_batch_size.default_value),
    'steps_per_epoch': str(train_steps_per_epoch.default_value),
    'validation_batch_size': str(validation_batch_size.default_value),
    'validation_steps': str(validation_steps_per_epoch.default_value),
    'optimizer': str(optimizer.default_value)
}

In [365]:
metric_definitions = [{'Name':'train:loss','Regex':'loss: ([0-9\\.]+)'},
                                    {'Name':'train:accuracy','Regex':'acc: ([0-9\\.]+)'},
                                    {'Name':'validation:loss','Regex':'val_loss: ([0-9\\.]+)'},
                                    {'Name':'validation:accuracy','Regex':'val_acc: ([0-9\\.]+)'}]

In [366]:
from sagemaker.huggingface import HuggingFace

estimator = HuggingFace(
        entry_point="train.py",
        source_dir = "./src/",
        role=role,
        instance_count=train_instance_count.default_value,
        volume_size = 50,
        max_run = 18000,
        instance_type=train_instance_type.default_value,
        transformers_version = "4.4",
        tensorflow_version  = "2.4",
        py_version="py37",
        input_mode = input_mode.default_value,
        hyperparameters = hyperparameters,
        metric_definitions = metric_definitions,
        enable_sagemaker_metrics = True
    )



In [367]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="PT1H") # PT1H represents `one hour`

In [368]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name='Train',
    estimator=estimator,
    inputs={
        'train': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'processed-train'
            ].S3Output.S3Uri
        ),
        'validation': TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                'processed-validation'
            ].S3Output.S3Uri
        )
    },
    cache_config=cache_config
)

print(training_step)

TrainingStep(name='Train', step_type=<StepTypeEnum.TRAINING: 'Training'>, depends_on=None)


## Evaluation Step

In [369]:
from sagemaker.sklearn.processing import SKLearnProcessor,ScriptProcessor

evaluation_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={'AWS_DEFAULT_REGION': region},
    max_runtime_in_seconds=7200
)

In [370]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='metrics',
    path='evaluation.json'
)

In [371]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

evaluation_step = ProcessingStep(
    name='EvaluateModel',
    processor=evaluation_processor,
    code='./src/evaluate_model_metrics.py',
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/input/model'
        ),
        ProcessingInput(
            source=processing_step.properties.ProcessingOutputConfig.Outputs['processed-test'].S3Output.S3Uri,
            destination='/opt/ml/processing/input/data'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='metrics', 
                         s3_upload_mode='EndOfJob',
                         source='/opt/ml/processing/output/metrics/'),
    ],
    job_arguments=[
        '--max_len', str(max_seq_length.default_value)
    ],
    property_files=[evaluation_report],
)

## Register model step

In [372]:
# 3 parameters
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

deploy_instance_type = ParameterString(
    name="DeployInstanceType",
    default_value="ml.m5.large"
)

deploy_instance_count = ParameterInteger(
    name="DeployInstanceCount",
    default_value=1
)

In [373]:
model_package_group_name = f"BaseBERT-Injury-Coding-{timestamp}"

print(model_package_group_name)

BaseBERT-Injury-Coding-1629753851


In [374]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            evaluation_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)

print(model_metrics)

Define deployment image for inference.

In [375]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4.1",
    py_version="py37",
    instance_type=deploy_instance_type,
    image_scope="inference"
)
print(inference_image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.4.1-cpu


### Register model

In [376]:
from sagemaker.workflow.step_collections import RegisterModel

register_step = RegisterModel(
    name="RegisterModel",
    estimator=estimator,
    image_uri=inference_image_uri, # Replace None 
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/jsonlines"],
    response_types=["application/jsonlines"],
    inference_instances=[deploy_instance_type],
    transform_instances=[deploy_instance_type], # batch transform is not used in this lab
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics
)

## Deployment Step

In [377]:
from sagemaker.model import Model

model_name = 'bert-model-{}'.format(timestamp)

model = Model(
    name=model_name,
    image_uri=inference_image_uri, # Replace None
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [378]:
from sagemaker.inputs import CreateModelInput

create_inputs = CreateModelInput(
    instance_type=deploy_instance_type, 
)

In [379]:
from sagemaker.workflow.steps import CreateModelStep

create_step = CreateModelStep(
    name="CreateModel",
    model=model, # Replace None
    inputs=create_inputs, # Replace None
)

# 6. Check accuracy condition step

In [380]:
min_accuracy_value = ParameterFloat(
    name="MinAccuracyValue",
    default_value=0.85 # random choice from three classes
)

In [381]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

minimum_accuracy_condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value",
    ),
    right=min_accuracy_value # minimum accuracy threshold
)

minimum_accuracy_condition_step = ConditionStep(
    name="AccuracyCondition",
    conditions=[minimum_accuracy_condition],
    if_steps=[register_step, create_step], # successfully exceeded or equaled the minimum accuracy, continue with model registration
    else_steps=[], # did not exceed the minimum accuracy, the model will not be registered
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# 7. Create pipeline

In [382]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[ 
        input_data,        
        processing_instance_count, 
        processing_instance_type, 
        max_seq_length, 
        is_sample_dataset, 
        transformer_model, 
        train_percentage,   
        epochs, 
        num_records, 
        learning_rate, 
        optimizer, 
        train_batch_size, 
        train_steps_per_epoch,  
        validation_batch_size, 
        validation_steps_per_epoch, 
        input_mode, 
        train_instance_count, 
        train_instance_type,   
        min_accuracy_value, 
        model_approval_status, 
        deploy_instance_type, 
        deploy_instance_count 
    ],
    steps=[processing_step, training_step, evaluation_step, minimum_accuracy_condition_step],
    sagemaker_session=sagemaker_session,
)


In [383]:
import json
from pprint import pprint

definition = json.loads(pipeline.definition())

pprint(definition)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Metadata': {},
 'Parameters': [{'DefaultValue': 's3://sagemaker-us-east-1-979294212144/injury-data/raw',
                 'Name': 'InputData',
                 'Type': 'String'},
                {'DefaultValue': 1,
                 'Name': 'ProcessingInstanceCount',
                 'Type': 'Integer'},
                {'DefaultValue': 'ml.c5.2xlarge',
                 'Name': 'ProcessingInstanceType',
                 'Type': 'String'},
                {'DefaultValue': 45, 'Name': 'MaxSeqLength', 'Type': 'Integer'},
                {'DefaultValue': 'True',
                 'Name': 'SampleDataset',
                 'Type': 'String'},
                {'DefaultValue': 'bert-base-uncased',
                 'Name': 'TransformerModel',
                 'Type': 'String'},
                {'DefaultValue': 1.0,
                 'Name': 'TrainPercentage',
                 'Type': 'Float'},
                {'DefaultValue': 3, 'Name': 'Epochs', 'Type': 'Integer'},
                {'DefaultValue'

In [384]:
response = pipeline.create(role_arn=role)

#pipeline_arn = response["PipelineArn"]
#print(pipeline_arn)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


In [385]:
execution = pipeline.start(
    parameters=dict(
        InputData=input_data_train,
        ProcessingInstanceCount=1,
        ProcessingInstanceType='ml.c5.2xlarge',
        MaxSeqLength=45,
        SampleDataset='False',
        TransformerModel = 'bert-based-uncased',
        TrainPercentage=1,
        Epochs=1,
        NumRecords = 138549,
        LearningRate=5e-5,
        optimizer = 'Adam',
        TrainBatchSize=16,
        TrainStepsPerEpoch=8659,
        ValidationBatchSize=16,
        ValidationStepsPerEpoch=962,
        InputMode= 'File',
        TrainInstanceCount=1,
        TrainInstanceType='ml.p3.2xlarge',
        MinAccuracyValue=0.85,
        ModelApprovalStatus='PendingManualApproval', 
        DeployInstanceType='ml.m5.large',
        DeployInstanceCount=1 
    )
)

print(execution.arn)

arn:aws:sagemaker:us-east-1:979294212144:pipeline/basebert-injury-coding-pipeline-1629753851/execution/0iq40kxccjvs
